In [2]:
import pandas as pd
import numpy as np
import re
import jieba
from wordcloud import WordCloud
from PIL import Image
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

In [3]:
data = pd.read_csv('C:/Users/12968/Desktop/数据科学实战-stock prediction/数据/新浪公司研报.csv')
data.head()

,_id,cate,content,date,ins,person,title
0,60715ab0a68de4a0c42408aa,公司,事件： \r\n 公司发布2021 年第一季度业绩预增公告。公司2021...,2021-04-09,上海申银万国证券研究所有限公司,张雷/陈明雨/黄华栋,东方电缆(603606)：完善产业布局 海缆及海工业务高速增长
1,60715ab2a68de4a0c42408af,公司,2020 年营收基本与2019 年持平，现金分红比例高达123%2020 年营收约1...,2021-04-09,浙商证券股份有限公司,王华君/李锋,美亚光电(002690)：2020年现金分红比例高达123%；2021年Q1高增长可期
2,60715ab5a68de4a0c42408b1,公司,事件 \r\n 2020 年报：公司发布2020 年年报以及2021 年...,2021-04-09,中信建投证券股份有限公司,郑勇,金禾实业(002597)：食品添加剂持续量增巩固龙头地位 资本开支大幅上行蓄力未来
3,60715ac1a68de4a0c42408b8,公司,事件 \r\n 公司发布2020 年年报：2020 年合计实现营业收入1...,2021-04-09,中信建投证券股份有限公司,郑勇/邓天泽,梅花生物(600873)：以量补价业绩稳健 21年业绩可期集中释放
4,60715ac1a68de4a0c42408ba,公司,公司20 年实现收入114.52 亿元，同比+29.34%；归母净利润8.38 亿元...,2021-04-09,招商证券股份有限公司,刘荣/时文博,杭叉集团(603298)：精益管理消化原材料价格影响 尽显龙头优势


In [4]:
# 一个例子
sample = data['content'][4]
sample = re.sub('\xa0\xa0\xa0\r\n','',sample)
sample = re.sub(' ','',sample)
sample

'公司20年实现收入114.52亿元，同比+29.34%；归母净利润8.38亿元，同比+29.99%；其中，20Q4实现收入32.43亿元，+54.14%；20Q4实现归母净利润2.11亿元，增长26.92%。精益化管理及规模效应使得公司在采购原材料及零部件环节溢价能力强，淡化原材料涨价带来的成本上升风险，Q4毛利率环比微降0.94pct，优于同行业竞争对手。杭叉20年综合净利率为8.1%，与去年持平，其中Q4净利率为7.42%。受益于制造业投资回升和物流仓储需求增长，行业景气度从3月开始迅速回暖，根据工程机械协会统计销量，行业20年销量同比+32%，杭叉销量同比+49%，增速远高于行业，略高于合力（+45%），杭叉国内市占率提升4%。从收入端看，全年收入增速29.34%，低于销量增速49%，一方面由于产品价格调整，另一方面是价值量较低的电动步行式仓储叉车占比提升。从利润端看，全年综合毛利率20.35%，净利率8.1%，较19年毛利率微降1个百分点，净利率持平。人民币升值影响较大，2020年汇兑损失3597万元，扣非归母净利润7.8亿元，同比+44%。扣除2020、2019年投资收益（2020：中策投资收益1.73亿、银行理财2645万元）影响，20年（扣非净利润-投资收益）同比+45.31%，反映了叉车的利润增长情况，处于行业领先地位。在20年行业价格竞争激烈，下半年原材料涨价，人民币升值的背景下，杭叉业绩超预期，体现优质龙头企业在上游议价、成本管理和规模化效应上的优势。叉车销量与PMI指数高度相关，且PMI指数具有一定前瞻性，当前PMI连续数月保持在50的容枯线上，叠加电动平衡重式叉车替换内燃平衡重式叉车，电动步行式仓储叉车的增量空间，杭叉销量有望保持高景气。基于行业景气及杭叉自身的成本控制能力，目前估值20倍左右相对合理。风险提示：原材料成本上涨盈利能力下降的风险；2021年下半年增速放缓。'

In [1]:
from nltk.tree import Tree
import logging
from stanfordcorenlp import StanfordCoreNLP

C:\Users\12968\Anaconda3\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
C:\Users\12968\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.o

In [2]:
nlp = StanfordCoreNLP(r'E:\py\stanford-corenlp-4.2.0',lang='zh',quiet=False,logging_level=logging.DEBUG)
res = nlp.parse('浙江大学有七个校区。')
print(res)
print(type(res))
nlp.close()

INFO:root:Initializing native server...
INFO:root:java -Xmx4g -cp "E:\py\stanford-corenlp-4.2.0\*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000
INFO:root:Server shell PID: 23220
INFO:root:The server is available.
INFO:root:{'properties': "{'annotators': 'pos,parse', 'outputFormat': 'json'}", 'pipelineLanguage': 'zh'}
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 127.0.0.1:9000
DEBUG:urllib3.connectionpool:http://127.0.0.1:9000 "POST /?properties=%7B%27annotators%27%3A+%27pos%2Cparse%27%2C+%27outputFormat%27%3A+%27json%27%7D&pipelineLanguage=zh HTTP/1.1" 200 5342
INFO:root:Cleanup...
INFO:root:Killing pid: 23664, cmdline: ['java', '-Xmx4g', '-cp', 'E:\\py\\stanford-corenlp-4.2.0\\*', 'edu.stanford.nlp.pipeline.StanfordCoreNLPServer', '-port', '9000']
INFO:root:Killing shell pid: 23220, cmdline: ['C:\\WINDOWS\\system32\\cmd.exe', '/c', 'java -Xmx4g -cp E:\\py\\stanford-corenlp-4.2.0\\* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000']


(ROOT
  (IP
    (NP (NR 浙江) (NN 大学))
    (VP (VE 有)
      (NP
        (QP (CD 七)
          (CLP (M 个)))
        (NP (NN 校区))))
    (PU 。)))
<class 'str'>


In [4]:
res

'(ROOT\r\n  (IP\r\n    (NP (NR 浙江) (NN 大学))\r\n    (VP (VE 有)\r\n      (NP\r\n        (QP (CD 七)\r\n          (CLP (M 个)))\r\n        (NP (NN 校区))))\r\n    (PU 。)))'

In [7]:
res.split("\r\n")

['(ROOT',
 '  (IP',
 '    (NP (NR 浙江) (NN 大学))',
 '    (VP (VE 有)',
 '      (NP',
 '        (QP (CD 七)',
 '          (CLP (M 个)))',
 '        (NP (NN 校区))))',
 '    (PU 。)))']

In [8]:
Tree.fromstring(res).draw()

In [ ]:
# 计算节点数-计算（的个数
result = []
for i in res:
    if i == '(':
        result += 1

INFO:root:Initializing native server...
INFO:root:java -Xmx4g -cp "E:\py\stanford-corenlp-4.2.0\*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9003
INFO:root:Server shell PID: 21856
INFO:root:The server is available.
INFO:root:{'properties': "{'annotators': 'pos,parse', 'outputFormat': 'json'}", 'pipelineLanguage': 'zh'}
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 127.0.0.1:9003
DEBUG:urllib3.connectionpool:http://127.0.0.1:9003 "POST /?properties=%7B%27annotators%27%3A+%27pos%2Cparse%27%2C+%27outputFormat%27%3A+%27json%27%7D&pipelineLanguage=zh HTTP/1.1" 200 2399


KeyboardInterrupt: 

In [1]:
sentence = '我叫小米'
with StanfordCoreNLP(r'E:\py\stanford-corenlp-4.2.0', lang='zh') as nlp:
    Tree.fromstring(nlp.parse(sentence)).draw()

NameError: name 'StanfordCoreNLP' is not defined

In [10]:
from nltk.tree import Tree
import re
import numpy as np
import logging
from stanfordcorenlp import StanfordCoreNLP
import random
import pandas as pd


class ParseTree:

    def __init__(self, text):
        self.text = text  # 传入的文本
        # 定义模型
        # self.nlp = StanfordCoreNLP(r'E:\py\stanford-corenlp-4.2.0', lang='zh', quiet=False, logging_level=logging.DEBUG)
        # 分句
        self.sentences = self.preprocess()

    def preprocess(self):
        """
        将整篇文本处理为一个列表，每个元素是文本里的句子
        :return:返回列表
        """
        sentences = []
        self.text = re.sub(' ', '', re.sub('\xa0\xa0\xa0\r\n', '', self.text))
        start = 0
        for i in range(len(self.text)):
            if self.text[i] in ['。', '!', '；', '？', '……', '，']:
                sentences.append(self.text[start:i + 1])
                start = i + 1
        return sentences

    def tree(self, sentence):
        nlp = StanfordCoreNLP(r'E:\py\stanford-corenlp-4.2.0', lang='zh', quiet=False, logging_level=logging.DEBUG)
        res = nlp.parse(sentence)
        # nlp.close()
        return res

    def sum_of_heights(self):
        """
        计算整篇文本的每句话构成的语法分析树的高度之和
        :return: 高度之和
        """
        sumHeights = []
        for sentence in self.sentences:
            res = self.tree(sentence)  # 语法树，是个字符串
            sum.append(len(res.split("\r\n")))
        return np.sum(sumHeights)

    def no_less_than_16(self):
        """
        计算整篇文本的高度不大于16的语法分析树的个数
        :return:
        """
        num = 0
        for sentence in self.sentences:
            res = self.tree(sentence)
            if len(res) >= 16:
                num += 1
        return num

    def nodes_sum(self):
        node_sums = []
        for sentence in self.sentences:
            res = self.nlp.parse(sentence)
            result = -1  # 去除root
            for i in res:
                if i == '(':
                    result += 1
            node_sums.append(result)
        return np.sum(node_sums)


if __name__ == '__main__':
    data = pd.read_csv('C:/Users/12968/Desktop/数据科学实战-stock prediction/数据/新浪公司研报.csv')
    sample = list(data['content'])[4]
    tree = ParseTree(sample)
    print('结果为'+str(tree.no_less_than_16()))

INFO:root:Initializing native server...
INFO:root:java -Xmx4g -cp "E:\py\stanford-corenlp-4.2.0\*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9011
INFO:root:Server shell PID: 22676
INFO:root:The server is available.
INFO:root:{'properties': "{'annotators': 'pos,parse', 'outputFormat': 'json'}", 'pipelineLanguage': 'zh'}
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 127.0.0.1:9011
DEBUG:urllib3.connectionpool:http://127.0.0.1:9011 "POST /?properties=%7B%27annotators%27%3A+%27pos%2Cparse%27%2C+%27outputFormat%27%3A+%27json%27%7D&pipelineLanguage=zh HTTP/1.1" 200 6152
INFO:root:Initializing native server...
INFO:root:java -Xmx4g -cp "E:\py\stanford-corenlp-4.2.0\*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9013
INFO:root:Server shell PID: 22920
INFO:root:The server is available.
INFO:root:{'properties': "{'annotators': 'pos,parse', 'outputFormat': 'json'}", 'pipelineLanguage': 'zh'}
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 127.

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [3]:
from nltk.tree import Tree
import logging
import pandas as pd
import re
from stanfordcorenlp import StanfordCoreNLP
data = pd.read_csv('C:/Users/12968/Desktop/数据科学实战-stock prediction/数据/新浪公司研报.csv')
nlp = StanfordCoreNLP(r'E:\py\stanford-corenlp-4.2.0',lang='zh',quiet=False,logging_level=logging.DEBUG)
sample = list(data['content'])[3:4][0]
# for sample in samples:
sentences = []
text = re.sub(' ', '', re.sub('\xa0\xa0\xa0\r\n', '', sample))
print('-------------------sample--------------')
print(sample)
print('-----------------------------------------')
start = 0
for i in range(len(text)):
    if text[i] in ['。', '!', '；', '？', '……', '：']:
        sentences.append(text[start:i + 1])
        start = i + 1
print('-------------------sentences--------------')
print(sentences)
print('-----------------------------------------')

INFO:root:Initializing native server...
INFO:root:java -Xmx4g -cp "E:\py\stanford-corenlp-4.2.0\*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9008
INFO:root:Server shell PID: 19624
INFO:root:The server is available.


-------------------sample--------------
    事件    
    公司发布2020 年年报：2020 年合计实现营业收入170.50 亿元，实现同比增长17.15%，实现归母净利9.82 亿元，同比减少0.67%。    
    对应Q4 单季净利润1.63 亿元，环比下降11%，同比增长22%。    
    简评    
    苏、赖氨酸价格改善，产品产销明显增加，量价齐升支撑业绩产品价格方面，味精、苏氨酸、赖氨酸在2020 年平均价格分别为7430、8386、7502，分别同比-722、+1033、+983 元/吨，整体有增有减，但原料玉米价格在2020 年仍有明显涨价，综合来看公司产品毛利率仍然有所下行。    
    产品产销量方面，白城基地40 万吨赖氨酸19 年投产、25 万吨味精20 年投产，为公司20 年贡献产能增量；同时，公司克服疫情影响，生产经营基本稳定。主要产品产销量因此均有明显增长，动物营养氨基酸产品（苏、赖氨酸等）销量增加16.43%，食品味觉形状优化产品（味精等）销量增加9.57%。以量补价，公司因此实现2020 年净利基本稳定。    
    费用率方面，公司全年销售、管理、财务费用率分别为2.16%、3.85%、0.83%，同比分别-5.87、-0.28、-1.03pct，其中销售费用率下降系会计政策变更，运费调整至营业成本项目，对公司利润无影响。    
    利润分配方面，公司预计分派现金红利9.3 亿元，占2020 年全年归母净利的94.70%，保持了一贯的高股息政策，以当前市值计算，股息率也达到4.95%。    
    苏赖氨酸Q1 大幅提价，21 年公司业绩有望集中释放味精及氨基酸，从长期逻辑看，行业竞争仍将从无序变为有序，行业价格中枢有望稳步提高。2021 年以来，受到需求回暖、玉米涨价支撑成本、企业协同等因素驱动，连续上调氨基酸产品报价，赖氨酸、苏氨酸市场价连续上行、价差也大幅走扩。Q1，苏氨酸、赖氨酸平均价差分别为8310、7485 元/吨，分别较2020 年大幅提升6452、4572 元/吨，公司产品实际交易价涨幅虽未必达到报价涨幅，但Q1 起业绩仍有望集中释放。    
    另一方面，内蒙能耗新政出台以来，当地味精、氨基酸有停止报价，后续如有限产或将

In [4]:
sentence = sentences[0]
print(sentence)
res = nlp.parse(sentence)
print(res)

INFO:root:{'properties': "{'annotators': 'pos,parse', 'outputFormat': 'json'}", 'pipelineLanguage': 'zh'}
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 127.0.0.1:9008
DEBUG:urllib3.connectionpool:http://127.0.0.1:9008 "POST /?properties=%7B%27annotators%27%3A+%27pos%2Cparse%27%2C+%27outputFormat%27%3A+%27json%27%7D&pipelineLanguage=zh HTTP/1.1" 200 4712


(ROOT
  (IP
    (NP (NN 事件) (NN 公司))
    (VP (VV 发布)
      (NP
        (NP (NT 2020年))
        (NP (NN 年报))
        (PU ：)))))


In [9]:
sentence = sentences[1]
print(sentence)
res = nlp.parse(sentence)
print(res)

INFO:root:{'properties': "{'annotators': 'pos,parse', 'outputFormat': 'json'}", 'pipelineLanguage': 'zh'}
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 127.0.0.1:9008
DEBUG:urllib3.connectionpool:http://127.0.0.1:9008 "POST /?properties=%7B%27annotators%27%3A+%27pos%2Cparse%27%2C+%27outputFormat%27%3A+%27json%27%7D&pipelineLanguage=zh HTTP/1.1" 200 6233


2020年合计实现营业收入170.50亿元，
(ROOT
  (IP
    (NP (NT 2020年))
    (VP
      (ADVP (AD 合计))
      (VP (VV 实现)
        (NP (NN 营业) (NN 收入))
        (QP (CD 170.50亿)
          (CLP (M 元)))))
    (PU ，)))


In [10]:
sentence = sentences[2]
print(sentence)
res = nlp.parse(sentence)
print(res)

INFO:root:{'properties': "{'annotators': 'pos,parse', 'outputFormat': 'json'}", 'pipelineLanguage': 'zh'}
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 127.0.0.1:9008
DEBUG:urllib3.connectionpool:http://127.0.0.1:9008 "POST /?properties=%7B%27annotators%27%3A+%27pos%2Cparse%27%2C+%27outputFormat%27%3A+%27json%27%7D&pipelineLanguage=zh HTTP/1.1" 500 36


实现同比增长17.15%，


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [11]:
sentence = sentences[3]
print(sentence)
res = nlp.parse(sentence)
print(res)

INFO:root:{'properties': "{'annotators': 'pos,parse', 'outputFormat': 'json'}", 'pipelineLanguage': 'zh'}
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 127.0.0.1:9008
DEBUG:urllib3.connectionpool:http://127.0.0.1:9008 "POST /?properties=%7B%27annotators%27%3A+%27pos%2Cparse%27%2C+%27outputFormat%27%3A+%27json%27%7D&pipelineLanguage=zh HTTP/1.1" 200 4684


实现归母净利9.82亿元，
(ROOT
  (IP
    (VP (VV 实现)
      (NP (NN 归母) (NN 净利))
      (QP (CD 9.82亿)
        (CLP (M 元))))
    (PU ，)))


In [12]:
sentence = sentences[4]
print(sentence)
res = nlp.parse(sentence)
print(res)

INFO:root:{'properties': "{'annotators': 'pos,parse', 'outputFormat': 'json'}", 'pipelineLanguage': 'zh'}
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 127.0.0.1:9008
DEBUG:urllib3.connectionpool:http://127.0.0.1:9008 "POST /?properties=%7B%27annotators%27%3A+%27pos%2Cparse%27%2C+%27outputFormat%27%3A+%27json%27%7D&pipelineLanguage=zh HTTP/1.1" 500 36


同比减少0.67%。


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [1]:
res = nlp.parse('受到需求回暖、玉米涨价支撑成本、企业协同等因素驱动,连续上调氨基酸产品报价')
print(res)

NameError: name 'nlp' is not defined

In [ ]:
from nltk.tree import Tree
import logging
import pandas as pd
import re
from stanfordcorenlp import StanfordCoreNLP
data = pd.read_csv('C:/Users/12968/Desktop/数据科学实战-stock prediction/数据/新浪公司研报.csv')
nlp = StanfordCoreNLP(r'E:\py\stanford-corenlp-4.2.0',lang='zh',quiet=False,logging_level=logging.DEBUG)
sample = list(data['content'])[3:4][0]
sentences = []
text = re.sub(' ', '', re.sub('\xa0\xa0\xa0\r\n', '', sample))
start = 0
for i in range(len(text)):
    if text[i] in ['。', '!', '；', '？', '……', '：']:
        sentences.append(text[start:i + 1])
        start = i + 1

# 给定sentences

for sentence in sentences:
    num = 0
    res = nlp.parse(sentence)
    for i in res:
        if 'NP' in i:
            num += 1